In [1]:
# Fungsi untuk tim Bajigur Cloth adalah fungsi untuk dapat meng-ekstraksi Feature-Feature yang ada pada review yang diberikan oleh customer,
# Hal ini diperlukan untuk membantu fungsi selanjutnya yaitu fungsi recommender sehingga tim Bajigur Cloth mudah menyimpulkan apakah produk layak dijadikan sebagai produk rekomendasi selanjutnya

# Fungsi Feature Extraction

# Data yang digunakan sebagai contoh adalah data film
import pandas as pd
import io
from google.colab import files

# Load jika menggunakan google colab
# uploaded = files.upload()
# metadata = pd.read_csv(io.BytesIO(uploaded['movies_metadata.csv']))

# Load jika data berada di satu directory yang sama
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
metadata = metadata.sample(n=100, random_state=42)
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
43526,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",https://www.netflix.com/title/80164212,411405,tt5717492,en,Small Crimes,"A disgraced former cop, fresh off a six-year p...",7.219022,/z7jmLmrs0pLlDU4GI6ItaJeqlET.jpg,"[{'name': 'Rooks Nest Entertainment', 'id': 34...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",2017-04-28,0.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Small Crimes,False,5.8,55.0
6383,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,42492,tt0069449,en,Up the Sandbox,"A young wife and mother, bored with day-to-day...",0.13845,/zwOmdqvPObv9EsMgne7EBYzPYGW.jpg,"[{'name': 'Barwood Films', 'id': 3645}, {'name...","[{'iso_3166_1': 'US', 'name': 'United States o...",1972-12-21,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Up the Sandbox,False,7.3,2.0
3154,False,NaN,1000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,12143,tt0103759,en,Bad Lieutenant,"While investigating a young nun's rape, a corr...",6.417037,/oe8VjWCKXktqA19T1ZWtaSn8rc2.jpg,"[{'name': 'Bad Lt. Productions', 'id': 11264}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1992-09-16,2019469.0,96.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Gambler. Thief. Junkie. Killer. Cop.,Bad Lieutenant,False,6.9,162.0
10146,False,NaN,0,"[{'id': 27, 'name': 'Horror'}, {'id': 10749, '...",NaN,9976,tt0380687,en,Satan's Little Helper,A naïve young boy unknowingly becomes the pawn...,2.233189,/59HgSvjDiaqUSYz43VZnpChgBkp.jpg,"[{'name': 'Intrinsic Value Films', 'id': 2828}...","[{'iso_3166_1': 'US', 'name': 'United States o...",2004-01-01,0.0,100.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,You'll laugh 'til you die,Satan's Little Helper,False,5.0,42.0
9531,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,46761,tt0157044,fr,Sitcom,The adventures of an upper-class suburban fami...,1.800582,/fZqanl6sCC29zU1K2v2mpuWyyYw.jpg,"[{'name': 'Fidélité Productions', 'id': 147}]","[{'iso_3166_1': 'FR', 'name': 'France'}]",1998-05-27,0.0,80.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Sitcom,False,6.4,27.0


In [2]:
# Langkah awal adalah menghilang kata tidak bisa kita jadikan sebagai feature yaitu seperti (dan, yang , ...)
# Import fungsi yang sudah bisa kita pakai dari 
from sklearn.feature_extraction.text import TfidfVectorizer
# Jika bahasa inggris
tfidf = TfidfVectorizer(stop_words='english')

# Jika bahasa indonesia
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# tfidf = TfidfVectorizer(StopWordRemoverFactory().get_stop_words())

# Hilangkan semua nilai null
metadata['review'] = metadata['overview'].fillna('')

#Mulai 
tfidf_matrix = tfidf.fit_transform(metadata['overview'].values.astype('U'))

#Output
tfidf_matrix.shape

(100, 1776)

In [12]:
# Langkah selanjutnya adalah mengihutung kesamaan antar data dengan feature yang sudah kita data
from sklearn.metrics.pairwise import linear_kernel

# Menggunakan cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

cosine_sim[:2]

array([[1.        , 0.        , 0.        , 0.        , 0.04380669,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.03304615, 0.        , 0.        , 0.02268336, 0.        ,
        0.07489186, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.13727903, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.03710582, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.05224136,
        0.        , 0.        , 0.0337046 , 0.03815746, 0.        ,
        0.        , 0.01304552, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.03286204, 0.        , 0.02181461, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [31]:
# Setelah semuanya selesai, kita memerlukan nama-nama produk yang sudah direview, sementara ini kita contohkan dengan data movie

indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

indices[:10]

title
Small Crimes               43526
Up the Sandbox              6383
Bad Lieutenant              3154
Satan's Little Helper      10146
Sitcom                      9531
Nightlight                 33663
Guernsey                   36733
The Bells                   3396
Due cuori, una cappella    30019
Freeheld                   33321
dtype: int64

In [33]:
# Fungsi yang dibutuhkan oleh customer adalah fungsi recommender dimana customer mendapatkan beberapa saran produk berdasarkan history review yang ia pernah lakukan
# Input produk yang sudah pernah customer beli sehingga fungsi dapat membandingkan kesamaan dengan produk lain, sementara ini kita contohkan dengan data movie

# Contoh data "Small Crimes"

def get_recommendations(title, cosine_sim=cosine_sim):
#    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]

    # Return 10 produk yang paling mirip
    return metadata['title'].iloc[movie_indices]

In [34]:
get_recommendations('Small Crimes')

4057                                The Unbelievable Truth
25969                                        Lady Gangster
36531                                                 Khel
40457                     How to Hook Up Your Home Theater
9531                                                Sitcom
36063                    The Staircase II: The Last Chance
31143                                       Deadly Daycare
10996    Marilyn Hotchkiss' Ballroom Dancing & Charm Sc...
22033                                         The Conclave
19819                                          The Year 01
Name: title, dtype: object